## Running the trained model on the test data

In [ ]:
import json
import jsonlines
from nltk import sent_tokenize
from transformers import pipeline, AutoTokenizer,AutoModelForSeq2SeqLM

Initializing model and tokenizer:

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('contest_train') #or any other path of yours
tokenizer = AutoTokenizer.from_pretrained('contest_train')

Opening the test data:

In [ ]:
data = []

with jsonlines.open('test.jsonl', 'r') as f:
    for line in f:
        data.append(line['translation']['ru'])

In [ ]:
data

Writing the preprocessing function:

In [ ]:
def get_translation(text, lang, cur_model, cur_tokenizer):
    if lang == 'ru':
        translator = pipeline("translation_en_to_ru", model=cur_model, tokenizer=cur_tokenizer)
    else:
        translator = pipeline("translation_ru_to_en", model=cur_model, tokenizer=cur_tokenizer)
    text_length = len(text.split())
    text_split = str(text).split()
    if text_length > 100:
        sentences = sent_tokenize(text)
        result = []
        for sent in sentences:
            result.append(translator(sent)[0]['translation_text'])
        result = ' '.join(result)

    else:
        translation = translator(text)
        result = translation[0]['translation_text']    

    return result

Running the translation:

In [ ]:
preds = []

for el in data:
    translated = get_translation(el, 'en', cur_model=model, cur_tokenizer=tokenizer)
    preds.append({"translation": {"ru": el, "en": translated}})

Saving the predictions into the desired format:

In [ ]:
with open('preds.json', 'w') as f:
      json.dumps(preds)